In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

# paths to each season's data
season_paths = [path/'2016-17', 
                path/'2017-18', 
                path/'2018-19',
                path/'2019-20',
                path/'2020-21']

# names for each season
season_names = ['1617', '1718', '1819', '1920', '2021']

# team codes
teams = pd.read_csv(path/'teams.csv')

all_players = build_players(path, season_paths, season_names, teams)

In [3]:
# players dataset
all_players.head()

,full_name,play_proba_1617,position_1617,id_1617,cost_1617,team_1617,web_name_1617,play_proba_1718,position_1718,id_1718,...,id_1920,cost_1920,team_1920,web_name_1920,play_proba_2021,position_2021,id_2021,cost_2021,team_2021,web_name_2021
0,David_Ospina,100,1.0,1.0,47.0,3.0,Ospina,100,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Petr_Cech,100,1.0,2.0,54.0,3.0,Cech,100,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Laurent_Koscielny,0,2.0,3.0,61.0,3.0,Koscielny,0,2.0,4.0,...,7.0,50.0,3.0,Koscielny,NaN,NaN,NaN,NaN,NaN,NaN
3,Per_Mertesacker,100,2.0,4.0,48.0,3.0,Mertesacker,100,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gabriel Armando_de Abreu,75,2.0,5.0,48.0,3.0,Gabriel,0,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# check columns are pandas series
type(all_players.loc[:,'id_1617'])

pandas.core.series.Series

In [5]:
all_players.shape

(1093, 16)

In [6]:
# teams dataset
teams

,team,team_code,team_1617,team_1718,team_1819,team_1920
0,Arsenal,3,1.0,1.0,1.0,1.0
1,Bournemouth,91,2.0,2.0,2.0,3.0
2,Burnley,90,3.0,4.0,4.0,5.0
3,Chelsea,8,4.0,5.0,6.0,6.0
4,Crystal Palace,31,5.0,6.0,7.0,7.0
5,Everton,11,6.0,7.0,8.0,8.0
6,Hull City,88,7.0,NaN,NaN,NaN
7,Leicester City,13,8.0,9.0,11.0,9.0
8,Liverpool,14,9.0,10.0,12.0,10.0
9,Manchester City,43,10.0,11.0,13.0,11.0


In [7]:
# create market value dataset for each season and combine
teams_mv = pd.concat([build_season_mv('1617', 10, range(12,32)),
                      build_season_mv('1718', 10, range(12,32)),
                      build_season_mv('1819', 10, range(12,32))], 
                     axis=0, ignore_index=True, sort=False)

ConnectionError: HTTPSConnectionPool(host='www.transfermarkt.com', port=443): Max retries exceeded with url: /premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2016 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000000000988C188>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed'))

In [7]:
train_df = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})
train_df = train_df[train_df['season'] != '1920']

In [8]:
teams_mv = train_df.groupby(['team', 'relative_market_value_team', 'season']).count().reset_index().iloc[:,[0,1,2]]
teams_mv.columns = ['name', 'relative_market_value', 'season']

In [9]:
# check a season
teams_mv[teams_mv['season'] == '1819']

,name,relative_market_value,season
0,Arsenal,1.435552,1819
5,Bournemouth,0.418108,1819
7,Brighton and Hove Albion,0.417745,1819
10,Burnley,0.449442,1819
11,Cardiff City,0.183890,1819
14,Chelsea,2.540586,1819
15,Crystal Palace,0.634856,1819
18,Everton,1.039221,1819
21,Fulham,0.405284,1819
23,Huddersfield Town,0.273778,1819


In [10]:
# create training sets for each season
df_1617 = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv)
df_1718 = build_season(season_paths[1], season_names[1], all_players, teams, teams_mv)
df_1819 = build_season(season_paths[2], season_names[2], all_players, teams, teams_mv)

In [11]:
# size of each season
[x.shape for x in [df_1617, df_1718, df_1819]]

[(23679, 34), (22467, 34), (21790, 34)]

In [10]:
# size of each season
[x.shape for x in [df_1617, df_1718, df_1819]]

[(23679, 11), (22467, 11), (21790, 11)]

In [12]:
# join together into one dataframe
df_train = pd.concat([df_1617, df_1718, df_1819], ignore_index=True, axis=0)
#df_all = df_1617.append(df_1718).append(df_1819).reset_index(drop=True)

In [13]:
df_train.shape

(67936, 34)

In [14]:
df_train.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
67926,Yerry_Mina,38,2,90,Everton,Tottenham Hotspur,1.039221,1.983109,False,1,...,31788,2,2,2.0,-45,68,113,0,2019-05-12T14:00:00Z,1819
67927,Yohan_Benalouane,38,2,0,Leicester City,Chelsea,0.796050,2.540586,True,0,...,43770,0,0,0.0,-331,119,450,0,2019-05-12T14:00:00Z,1819
67928,Yoshinori_Muto,38,4,24,Newcastle United,Fulham,0.483921,0.405284,False,1,...,3344,4,0,0.0,11,47,36,0,2019-05-12T14:00:00Z,1819
67929,Younes_Kaboul,38,2,0,Watford,West Ham United,0.412301,0.749352,True,0,...,5304,4,1,0.0,-11,6,17,0,2019-05-12T14:00:00Z,1819
67930,Youri_Tielemans,38,3,90,Leicester City,Chelsea,0.796050,2.540586,True,3,...,106893,0,0,11.0,2939,10095,7156,0,2019-05-12T14:00:00Z,1819
67931,Yves_Bissouma,38,3,90,Brighton and Hove Albion,Manchester City,0.417745,2.540586,True,2,...,1736,4,1,0.0,30,73,43,0,2019-05-12T14:00:00Z,1819
67932,Zech_Medley,38,2,0,Arsenal,Burnley,1.435552,0.449442,False,0,...,5341,3,1,0.0,62,171,109,0,2019-05-12T14:00:00Z,1819
67933,Zeze Steven_Sessegnon,38,2,0,Fulham,Newcastle United,0.405284,0.483921,True,0,...,5780,4,0,0.0,125,332,207,0,2019-05-12T14:00:00Z,1819
67934,Álvaro_Morata,38,4,0,Chelsea,Leicester City,2.540586,0.796050,False,0,...,83088,0,0,0.0,-139,15,154,0,2019-05-12T14:00:00Z,1819
67935,Çaglar_Söyüncü,38,2,0,Leicester City,Chelsea,0.796050,2.540586,True,0,...,2405,0,0,0.0,-1,11,12,0,2019-05-12T14:00:00Z,1819


In [111]:
df_train[(df_train['team'] == 'Cardiff City') & (df_train['season'] == '1819') & (df_train['gw'] == 1)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
46173,Alex_Smithies,1,1,0,Cardiff City,Bournemouth,0.18389,0.418108,False,0,...,6692,0,2,0.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46194,Anthony_Pilkington,1,3,0,Cardiff City,Bournemouth,0.18389,0.418108,False,0,...,11915,0,2,0.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46197,Aron_Gunnarsson,1,3,0,Cardiff City,Bournemouth,0.18389,0.418108,False,0,...,66660,0,2,0.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46200,Ashley Darel Jazz_Richards,1,2,0,Cardiff City,Bournemouth,0.18389,0.418108,False,0,...,47860,0,2,0.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46224,Bobby_Reid,1,4,87,Cardiff City,Bournemouth,0.18389,0.418108,False,2,...,33401,0,2,35.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46228,Bruno_Ecuele Manga,1,2,90,Cardiff City,Bournemouth,0.18389,0.418108,False,1,...,5192,0,2,0.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46232,Callum_Paterson,1,3,62,Cardiff City,Bournemouth,0.18389,0.418108,False,2,...,8529,0,2,18.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46272,Danny_Ward_2,1,3,28,Cardiff City,Bournemouth,0.18389,0.418108,False,1,...,15437,0,2,0.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46278,David Junior_Hoilett,1,3,90,Cardiff City,Bournemouth,0.18389,0.418108,False,2,...,20590,0,2,8.0,0,0,0,0,2018-08-11T14:00:00Z,1819
46333,Gary_Madine,1,4,2,Cardiff City,Bournemouth,0.18389,0.418108,False,1,...,51829,0,2,0.0,0,0,0,0,2018-08-11T14:00:00Z,1819


In [16]:
# 'team' is from end of season but some players moved club mid-season
# so have incorrect 'team' for some rows
# get list of actual fixtures - games where there were at least 10 players
fixtures = df_train.groupby(['team', 'opponent_team', 'kickoff_time', 'season']).count()['player'][df_train.groupby(['team', 'opponent_team', 'kickoff_time', 'season']).count()['player'] > 10]
fixtures = fixtures.reset_index()[['team', 'opponent_team','kickoff_time', 'season']]
fixtures

,team,opponent_team,kickoff_time,season
0,Arsenal,Bournemouth,2016-11-27T14:15:00Z,1617
1,Arsenal,Bournemouth,2017-01-03T19:45:00Z,1617
2,Arsenal,Bournemouth,2017-09-09T14:00:00Z,1718
3,Arsenal,Bournemouth,2018-01-14T13:30:00Z,1718
4,Arsenal,Bournemouth,2018-11-25T13:30:00Z,1819
...,...,...,...,...
2275,Wolverhampton Wanderers,Tottenham Hotspur,2018-12-29T15:00:00Z,1819
2276,Wolverhampton Wanderers,Watford,2018-10-20T14:00:00Z,1819
2277,Wolverhampton Wanderers,Watford,2019-04-27T14:00:00Z,1819
2278,Wolverhampton Wanderers,West Ham United,2018-09-01T14:00:00Z,1819


In [17]:
# replace 'team' in df_train with 'team' from fixtures
# 'opponent_team', 'kickoff_time_formatted' and 'season' are unique
# join on these
df_train['team'] = df_train.merge(fixtures, on=['opponent_team', 'kickoff_time', 'season'], how='left')['team_y']

In [110]:
df_train

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron_Cresswell,1,2,0,West Ham United,Chelsea,0.895471,2.243698,False,0,...,14023,1,2,0.0,0,0,0,0,2016-08-15T19:00:00Z,1617
1,Aaron_Lennon,1,3,15,Everton,Tottenham Hotspur,1.057509,1.433690,True,1,...,13918,1,1,0.0,0,0,0,0,2016-08-13T14:00:00Z,1617
2,Aaron_Ramsey,1,3,60,Arsenal,Liverpool,1.944129,1.465860,True,2,...,163170,4,3,23.0,0,0,0,0,2016-08-14T15:00:00Z,1617
3,Abdoulaye_Doucouré,1,3,0,Watford,Southampton,0.704200,0.796805,False,0,...,1051,1,1,0.0,0,0,0,0,2016-08-13T14:00:00Z,1617
4,Abdul Rahman_Baba,1,2,0,Chelsea,West Ham United,2.243698,0.895471,True,0,...,1243,1,2,0.0,0,0,0,0,2016-08-15T19:00:00Z,1617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67931,Yves_Bissouma,38,3,90,Brighton and Hove Albion,Manchester City,0.417745,2.540586,True,2,...,1736,4,1,0.0,30,73,43,0,2019-05-12T14:00:00Z,1819
67932,Zech_Medley,38,2,0,Arsenal,Burnley,1.435552,0.449442,False,0,...,5341,3,1,0.0,62,171,109,0,2019-05-12T14:00:00Z,1819
67933,Zeze Steven_Sessegnon,38,2,0,Fulham,Newcastle United,0.405284,0.483921,True,0,...,5780,4,0,0.0,125,332,207,0,2019-05-12T14:00:00Z,1819
67934,Álvaro_Morata,38,4,0,Chelsea,Leicester City,2.540586,0.796050,False,0,...,83088,0,0,0.0,-139,15,154,0,2019-05-12T14:00:00Z,1819


In [112]:
# save latest training set to csv
df_train.to_csv(path/'initial_train_v2.csv')#, encoding='latin')

In [20]:
train_df = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})
train_df = train_df[train_df['season'] == '1920']
train_df

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
67936,Aaron_Cresswell,1,2,90,West Ham United,Manchester City,0.739196,2.727025,True,0,1920,1.0
67937,Aaron_Lennon,1,3,6,Burnley,Southampton,0.441799,0.624210,True,1,1920,1.0
67938,Aaron_Mooy,1,3,0,Brighton and Hove Albion,Watford,0.476156,0.555819,False,0,1920,NaN
67939,Aaron_Ramsdale,1,1,90,Bournemouth,Sheffield United,0.687124,0.150631,True,2,1920,1.0
67940,Aaron_Wan-Bissaka,1,2,90,Manchester United,Chelsea,1.679697,1.798870,True,8,1920,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
78969,Grant_Hanley,20,2,90,Norwich,Tottenham Hotspur,0.295212,2.001099,True,1,1920,1.0
78970,Roberto_Pereyra,20,3,0,Watford,Aston Villa,0.433797,0.522434,True,0,1920,1.0
78971,Sead_Kolasinac,20,2,0,Arsenal,Chelsea,1.413491,1.757609,True,0,1920,0.0
78972,Stuart_Armstrong,20,3,22,Southampton,Crystal Palace,0.469252,0.464142,True,1,1920,1.0


In [21]:
teams_mv = train_df.groupby(['team', 'season']).count().reset_index().iloc[:,[0,1]]
teams_mv.columns = ['name', 'season']
teams_mv['relative_market_value'] = 1
teams_mv

,name,season,relative_market_value
0,Arsenal,1920,1
1,Aston Villa,1920,1
2,Bournemouth,1920,1
3,Brighton and Hove Albion,1920,1
4,Burnley,1920,1
5,Chelsea,1920,1
6,Crystal Palace,1920,1
7,Everton,1920,1
8,Leicester City,1920,1
9,Liverpool,1920,1


In [22]:
# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

In [23]:
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, range(1, 21))

In [69]:
train_df = train_df[['player', 'gw', 'team', 'relative_market_value_team', 'relative_market_value_opponent_team', 'play_proba']]
train_df

,player,gw,team,relative_market_value_team,relative_market_value_opponent_team,play_proba
67936,Aaron_Cresswell,1,West Ham United,0.739196,2.727025,1.0
67937,Aaron_Lennon,1,Burnley,0.441799,0.624210,1.0
67938,Aaron_Mooy,1,Brighton and Hove Albion,0.476156,0.555819,NaN
67939,Aaron_Ramsdale,1,Bournemouth,0.687124,0.150631,1.0
67940,Aaron_Wan-Bissaka,1,Manchester United,1.679697,1.798870,1.0
...,...,...,...,...,...,...
78969,Grant_Hanley,20,Norwich,0.295212,2.001099,1.0
78970,Roberto_Pereyra,20,Watford,0.433797,0.522434,1.0
78971,Sead_Kolasinac,20,Arsenal,1.413491,1.757609,0.0
78972,Stuart_Armstrong,20,Southampton,0.469252,0.464142,1.0


In [55]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron_Cresswell,1,2,90,West Ham United,Manchester City,1,1,True,0,...,23399,5,0,0.0,0,0,0,0,2019-08-10T11:30:00Z,1920
1,Aaron_Lennon,1,3,6,Burnley,Southampton,1,1,True,1,...,8105,0,3,0.0,0,0,0,0,2019-08-10T14:00:00Z,1920
2,Aaron_Mooy,1,3,0,Brighton and Hove Albion,Watford,1,1,False,0,...,16261,3,0,0.0,0,0,0,0,2019-08-10T14:00:00Z,1920
3,Aaron_Ramsdale,1,1,90,Bournemouth,Sheffield United,1,1,True,2,...,3091,1,1,0.0,0,0,0,0,2019-08-10T14:00:00Z,1920
4,Aaron_Wan-Bissaka,1,2,90,Manchester United,Chelsea,1,1,True,8,...,1879259,0,4,2.0,0,0,0,0,2019-08-11T15:30:00Z,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11033,Grant_Hanley,20,2,90,Norwich,Tottenham Hotspur,1,1,True,1,...,144601,2,2,4.0,1237,3686,2449,0,2019-12-28T17:30:00Z,1920
11034,Roberto_Pereyra,20,3,0,Watford,Aston Villa,1,1,True,0,...,46937,0,3,0.0,83,619,536,0,2019-12-28T15:00:00Z,1920
11035,Sead_Kolasinac,20,2,0,Arsenal,Chelsea,1,1,True,0,...,38674,2,1,0.0,-1303,6,1309,0,2019-12-29T14:00:00Z,1920
11036,Stuart_Armstrong,20,3,22,Southampton,Crystal Palace,1,1,True,1,...,6223,1,1,2.0,300,385,85,0,2019-12-28T15:00:00Z,1920


In [71]:
df_new = df_latest.merge(train_df, on=['player', 'gw', 'team'], how='left')

In [84]:
list(range(6)) + list(range(8,38))

[0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37]

In [80]:
np.arange(8,38)

array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37])

In [88]:
df_new = df_new.iloc[:,list(range(6)) + list(range(8,37))]

In [109]:
df_new.columns

Index(['player', 'gw', 'position', 'minutes', 'team', 'opponent_team',
       'was_home', 'total_points', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'saves', 'selected', 'team_a_score', 'team_h_score',
       'threat', 'transfers_balance', 'transfers_in', 'transfers_out',
       'yellow_cards', 'kickoff_time', 'season', 'relative_market_value_team',
       'relative_market_value_opponent_team', 'play_proba'],
      dtype='object')

In [108]:
df_new.columns = ['player', 'gw', 'position', 'minutes', 'team', 'opponent_team',
       'was_home', 'total_points', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'saves', 'selected', 'team_a_score', 'team_h_score',
       'threat', 'transfers_balance', 'transfers_in', 'transfers_out',
       'yellow_cards', 'kickoff_time', 'season',
       'relative_market_value_team', 'relative_market_value_opponent_team',
       'play_proba']

In [120]:
df_train_v2 = pd.concat([df_train, df_new], ignore_index=True, sort=False, axis=0)

In [121]:
df_train_v2.to_csv(path/'train_v2.csv')

In [23]:
# save latest training set to csv
df_train.to_csv(path/'initial_train.csv')#, encoding='latin')

In [46]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [47]:
fixtures.head(10)

,home_team,away_team,gw
0,Liverpool,Norwich,1
1,West Ham United,Manchester City,1
2,Bournemouth,Sheffield United,1
3,Burnley,Southampton,1
4,Crystal Palace,Everton,1
5,Watford,Brighton and Hove Albion,1
6,Tottenham Hotspur,Aston Villa,1
7,Leicester City,Wolverhampton Wanderers,1
8,Newcastle United,Arsenal,1
9,Manchester United,Chelsea,1


In [48]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [49]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,1,Liverpool,Norwich,14,45
1,1,West Ham United,Manchester City,21,43
2,1,Bournemouth,Sheffield United,91,49
3,1,Burnley,Southampton,90,20
4,1,Crystal Palace,Everton,31,11


In [50]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [51]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [52]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,1,Norwich,Liverpool,Sam_Byram,2.0,45.0,None,False
1,1,Norwich,Liverpool,Tim_Krul,1.0,45.0,None,False
2,1,Norwich,Liverpool,Grant_Hanley,2.0,40.0,None,False
3,1,Norwich,Liverpool,Timm_Klose,2.0,45.0,0,False
4,1,Norwich,Liverpool,Ben_Godfrey,2.0,45.0,100,False


In [53]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,1,Liverpool,Norwich,Alex_Oxlade-Chamberlain,3.0,65.0,None,True
1,1,Liverpool,Norwich,Andrew_Robertson,2.0,70.0,None,True
2,1,Liverpool,Norwich,Simon_Mignolet,1.0,45.0,0,True
3,1,Liverpool,Norwich,Dejan_Lovren,2.0,55.0,None,True
4,1,Liverpool,Norwich,Joseph_Gomez,2.0,55.0,None,True


In [58]:
home_df.loc[home_df['play_proba'] == 'None', 'play_proba'] = 100

0       None
1       None
3       None
4       None
5       None
6       None
7       None
9       None
10      None
11      None
12      None
14      None
15      None
16      None
17      None
19      None
20      None
21      None
22      None
23      None
24      None
27      None
28      None
29      None
32      None
33      None
34      None
35      None
36      None
37      None
        ... 
9672    None
9673    None
9674    None
9676    None
9677    None
9679    None
9680    None
9681    None
9682    None
9683    None
9686    None
9687    None
9688    None
9691    None
9692    None
9693    None
9694    None
9695    None
9696    None
9698    None
9699    None
9700    None
9701    None
9702    None
9703    None
9704    None
9705    None
9706    None
9707    None
9708    None
Name: play_proba, Length: 7695, dtype: object

In [72]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# set minutes equal to 90 multiplied by their play probabiliby for all players, for now
remaining_season_df['minutes'] = 90 * remaining_season_df['play_proba']

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

In [76]:
remaining_season_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,1,Liverpool,Norwich,Alex_Oxlade-Chamberlain,3,6.5,1.0,True,2.291459,0.197772,1920,90.0
1,1,Liverpool,Norwich,Andrew_Robertson,2,7.0,1.0,True,2.291459,0.197772,1920,90.0
2,1,Liverpool,Norwich,Simon_Mignolet,1,4.5,0.0,True,2.291459,0.197772,1920,0.0
3,1,Liverpool,Norwich,Dejan_Lovren,2,5.5,1.0,True,2.291459,0.197772,1920,90.0
4,1,Liverpool,Norwich,Joseph_Gomez,2,5.5,1.0,True,2.291459,0.197772,1920,90.0


In [77]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')

In [1]:
# packages reguired for scraping fbref
import requests
from bs4 import BeautifulSoup
import re
import backoff

In [3]:
# various lists data fields to scrape
# standard stats
features = ['squad', 'comp', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct', 'passes_progressive_distance', 
            'carries', 'carry_progressive_distance', 'dribbles_completed', 'dribbles']

In [3]:
# functions to scrape fbref data
# get stats (given by features) for all players/teams
# get stats (given by features) for all players/teams
# for season pages given by season_urls
def get_stats(features, season_urls, url_base):
    seasons_list = []
    
    for url, season_name in zip(season_urls['url'], season_urls['season']):
        print(season_name)
        print(url)
        season_table = get_table(url)
        team_urls = get_urls(['squad'], 'squad', season_table, url_base)
        season_df = get_season_stats(features, team_urls, url_base)
        season_df.insert(0, 'season', season_name)
        seasons_list.append(season_df)
    
    df = pd.concat(seasons_list, ignore_index=True)
    return df

# get stats (given by features) for all players 
# for team pages given by team_urls
def get_season_stats(features, team_urls, url_base):
    teams_list = []
    
    for url, team_name in zip(team_urls['url'], team_urls['squad']):
        print(team_name)
        print(url)
        team_table = get_table(url)
        player_urls = get_urls(['player', 'games'], 'matches', team_table, url_base)
        team_df = get_team_stats(features, player_urls)
        teams_list.append(team_df)
        
    df = pd.concat(teams_list, ignore_index=True)
    return df

# get stats (given by features)
# for all players given by player_urls
def get_team_stats(features, player_urls):
    players_list = []
    player_urls_played = player_urls[player_urls['games'] != '0']

    for url, player_name in zip(player_urls_played['url'], player_urls_played['player']):
        print(player_name)
        player_table = get_table(url)
        player_df = get_player_stats(features, player_table)
        player_df.insert(0, 'player', player_name)
        if len(player_df) > 0:
            players_list.append(player_df[player_df['comp'] == 'Premier League'])

    df = pd.concat(players_list, ignore_index=True)
    return df

# get game by game player stats for one season
# date always included by default
def get_player_stats(features, player_table):
    pre_dict = dict()    
    table_rows = player_table.find_all('tr')    
    for row in table_rows:
        if(row.find('td',{"data-stat":'xg'}) != None):
            date = row.find('th',{"data-stat":'date'}).text.strip().encode().decode("utf-8")
            if date != '':
                if 'date' in pre_dict:
                    pre_dict['date'].append(date)
                else:
                    pre_dict['date'] = [date]
                for f in features:
                    text = row.find('td',{"data-stat":f}).text.strip().encode().decode("utf-8")
                    if f in pre_dict:
                        pre_dict[f].append(text)
                    else:
                        pre_dict[f] = [text]
    df = pd.DataFrame.from_dict(pre_dict)
    return df

def backoff_hdlr(details):
    print ("Backing off {wait:0.1f} seconds afters {tries} tries "
           "calling function {target} with args {args} and kwargs "
           "{kwargs}".format(**details))

# get a table on any page, defaults to first one
@backoff.on_exception(
    backoff.expo,
    (IndexError, requests.exceptions.RequestException),
    max_tries=5,
    on_backoff=backoff_hdlr
)
def get_table(url, table_no=0):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")    
    table = all_tables[table_no]
    return table

# get text and urls for a given field
def get_urls(text_fields, url_field, table, url_base):
    pre_dict = dict()    
    table_rows = table.find_all('tr')
    for row in table_rows:
        if(row.find('th',{"scope":"row"}) != None):
            for f in text_fields:
                if(row.find('th',{"data-stat":f}) != None):
                    text = row.find('th',{"data-stat":f}).text.strip().encode().decode("utf-8")
                else:
                    text = row.find('td',{"data-stat":f}).text.strip().encode().decode("utf-8")
                if f in pre_dict:
                    pre_dict[f].append(text)
                else: 
                    pre_dict[f] = [text]
                
            if row.find('th',{"data-stat":url_field}) != None:
                url = url_base + row.find('th',{"data-stat":url_field}).find('a').get('href')
#                 url = url_base + row.find('a')['href']
            else:
                url = url_base + row.find('td',{"data-stat":url_field}).find('a').get('href')

            if 'url' in pre_dict:
                pre_dict['url'].append(url)
            else: 
                pre_dict['url'] = [url]
                
    df = pd.DataFrame.from_dict(pre_dict)
    return df

In [6]:
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'

url = url_base + pl_history_page

pl_history_table = get_table(url)
# pl_history_df = 
get_urls(['season'], 'season', pl_history_table, url_base)

,season,url
0,2020-2021,https://fbref.com/en/comps/9/Premier-League-Stats
1,2019-2020,https://fbref.com/en/comps/9/3232/2019-2020-Pr...
2,2018-2019,https://fbref.com/en/comps/9/1889/2018-2019-Pr...
3,2017-2018,https://fbref.com/en/comps/9/1631/2017-2018-Pr...
4,2016-2017,https://fbref.com/en/comps/9/1526/2016-2017-Pr...
5,2015-2016,https://fbref.com/en/comps/9/1467/2015-2016-Pr...
6,2014-2015,https://fbref.com/en/comps/9/733/2014-2015-Pre...
7,2013-2014,https://fbref.com/en/comps/9/669/2013-2014-Pre...
8,2012-2013,https://fbref.com/en/comps/9/602/2012-2013-Pre...
9,2011-2012,https://fbref.com/en/comps/9/534/2011-2012-Pre...


In [7]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table
# enhanced stats only available from 2017/18 season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[:4]

df_enhanced = get_stats(features, season_urls, url_base)

2020-2021
https://fbref.com/en/comps/9/Premier-League-Stats
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Andrew Robertson
Roberto Firmino
Georginio Wijnaldum
Mohamed Salah
Sadio Mané
Fabinho
Alisson
Trent Alexander-Arnold
Jordan Henderson
Joël Matip
Curtis Jones
Joe Gomez
Naby Keïta
Diogo Jota
Virgil van Dijk
Takumi Minamino
James Milner
Adrián
Nathaniel Phillips
Neco Williams
Thiago Alcántara
Rhys Williams
Caoimhin Kelleher
Xherdan Shaqiri
Alex Oxlade-Chamberlain
Divock Origi
Everton
https://fbref.com/en/squads/d3fd31cc/Everton-Stats
Dominic Calvert-Lewin
Michael Keane
Abdoulaye Doucouré
Yerry Mina
Jordan Pickford
Allan
Richarlison
James Rodríguez
Ben Godfrey
Alex Iwobi
Gylfi Sigurðsson
Lucas Digne
Mason Holgate
Séamus Coleman
André Gomes
Tom Davies
Fabian Delph
Robin Olsen
Bernard
Anthony Gordon
Jonjoe Kenny
Niels Nkounkou
Cenk Tosun
Moise Kean
Theo Walcott
Leicester City
https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats
James Justin
Kasper Schmeichel
Youri 

Naby Keïta
Dejan Lovren
Joël Matip
Divock Origi
Adam Lallana
Neco Williams
Takumi Minamino
Xherdan Shaqiri
Curtis Jones
Harvey Elliot
Manchester City
https://fbref.com/en/squads/b8fd03ef/2019-2020/Manchester-City-Stats
Ederson
Kevin De Bruyne
Raheem Sterling
Rodri
Kyle Walker
Fernandinho
Bernardo Silva
David Silva
Gabriel Jesus
İlkay Gündoğan
Riyad Mahrez
Nicolás Otamendi
Benjamin Mendy
Sergio Agüero
Aymeric Laporte
Oleksandr Zinchenko
João Cancelo
John Stones
Phil Foden
Eric García
Angeliño
Claudio Bravo
Thomas Doyle
Leroy Sané
Manchester Utd
https://fbref.com/en/squads/19538871/2019-2020/Manchester-United-Stats
David de Gea
Harry Maguire
Victor Lindelöf
Aaron Wan-Bissaka
Marcus Rashford
Anthony Martial
Daniel James
Fred
Scott McTominay
Luke Shaw
Nemanja Matić
Andreas Pereira
Bruno Fernandes
Paul Pogba
Mason Greenwood
Brandon Williams
Ashley Young
Jesse Lingard
Juan Mata
Axel Tuanzebe
Timothy Fosu-Mensah
Phil Jones
Eric Bailly
Marcos Rojo
Diogo Dalot
Odion Ighalo
Tahith Chong
Angel Go

Max Aarons
Tim Krul
Teemu Pukki
Kenny McLean
Ben Godfrey
Todd Cantwell
Emi Buendía
Alexander Tettey
Jamal Lewis
Christoph Zimmermann
Sam Byram
Onel Hernández
Grant Hanley
Marco Stiepermann
Tom Trybull
Ondrej Duda
Ibrahim Amadou
Lukas Rupp
Moritz Leitner
Timm Klose
Mario Vrančić
Josip Drmić
Adam Idah
Michael McGovern
Ralf Fährmann
Dennis Srbeny
Josh Martin
Patrick Roberts
Akin Famewo
Jordan Thomas
2018-2019
https://fbref.com/en/comps/9/1889/2018-2019-Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats
Ederson
Aymeric Laporte
Bernardo Silva
Raheem Sterling
Sergio Agüero
Kyle Walker
David Silva
Fernandinho
İlkay Gündoğan
Leroy Sané
John Stones
Riyad Mahrez
Nicolás Otamendi
Oleksandr Zinchenko
Vincent Kompany
Kevin De Bruyne
Benjamin Mendy
Danilo
Gabriel Jesus
Fabian Delph
Phil Foden
Liverpool
https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats
Alisson
Virgil van Dijk
Mohamed Salah
Andrew Robertson
Sadio Mané
Georginio Wijnal

Fulham
https://fbref.com/en/squads/fd962109/2018-2019/Fulham-Stats
Aleksandar Mitrović
Sergio Rico
Denis Odoi
Calum Chambers
Joe Bryan
Jean Seri
Ryan Sessegnon
Maxime Le Marchand
Tim Ream
Tom Cairney
André Schürrle
Cyrus Christie
Andre-Frank Zambo Anguissa
Ryan Babel
Alfie Mawson
Luciano Vietto
Kevin McDonald
Timothy Fosu-Mensah
Marcus Bettinelli
Aboubakar Kamara
Stefan Johansen
Håvard Nordtveit
Floyd Ayité
Fabricio
Ibrahima Cissé
Neeskens Kebano
Lazar Marković
Harvey Elliot
Huddersfield
https://fbref.com/en/squads/f5922ca5/2018-2019/Huddersfield-Town-Stats
Christopher Schindler
Terence Kongolo
Jonas Lössl
Jonathan Hogg
Alex Pritchard
Aaron Mooy
Philip Billing
Mathias Jørgensen
Chris Löwe
Erik Durm
Florent Hadergjonaj
Steve Mounié
Juninho Bacuna
Elias Kachunga
Tommy Smith
Laurent Depoitre
Isaac Mbenza
Karlan Grant
Jon Gorenc Stanković
Ben Hamer
Adama Diakhaby
Rajiv van La Parra
Jason Puncheon
Danny Williams
Joel Coleman
Aaron Rowe
Abdelhamid Sabiri
Demeaco Duhaney
Ramadan Sobhi
Matty D

Laurent Depoitre
Elias Kachunga
Collin Quaner
Scott Malone
Alex Pritchard
Danny Williams
Terence Kongolo
Philip Billing
Joe Lolley
Abdelhamid Sabiri
Martin Cranie
Kasey Palmer
Dean Whitehead
Michael Hefele
Southampton
https://fbref.com/en/squads/33c895d4/2017-2018/Southampton-Stats
Ryan Bertrand
Oriol Romeu
Dušan Tadić
Cédric Soares
Wesley Hoedt
Maya Yoshida
Jack Stephens
Nathan Redmond
Fraser Forster
Mario Lemina
James Ward-Prowse
Pierre Højbjerg
Alex McCarthy
Steven Davis
Shane Long
Sofiane Boufal
Manolo Gabbiadini
Virgil van Dijk
Charlie Austin
Guido Carrillo
Jan Bednarek
Jérémy Pied
Matt Targett
Josh Sims
Sam McQueen
Michael Obafemi
Swansea City
https://fbref.com/en/squads/fb10988f/2017-2018/Swansea-City-Stats
Łukasz Fabiański
Alfie Mawson
Martin Olsson
Kyle Naughton
Jordan Ayew
Federico Fernández
Tom Carroll
Mike van der Hoorn
Sam Clucas
Ki Sung-yueng
Nathan Dyer
Tammy Abraham
Leroy Fer
André Ayew
Roque Mesa
Andy King
Renato Sanches
Wilfried Bony
Luciano Narsingh
Wayne Routledge
L

In [6]:
df_train = pd.read_csv(path/'train_v6.csv', index_col=0, dtype={'season':str})

In [73]:
enhanced_players =  df_enhanced['player'].unique().tolist()
standard_players = df_train['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

['Fabinho',
 'Alisson',
 'Joël Matip',
 'Joe Gomez',
 'Naby Keïta',
 'Adrián',
 'Thiago Alcántara',
 'Allan',
 'Richarlison',
 'Gylfi Sigurðsson',
 'Séamus Coleman',
 'André Gomes',
 'Bernard',
 'Çağlar Söyüncü',
 'Bruno Fernandes',
 'Fred',
 'Nemanja Matić',
 'Alex Telles',
 'Pierre Højbjerg',
 'Son Heung-min',
 'Lucas Moura',
 'Érik Lamela',
 'Dele Alli',
 'Carlos Vinícius',
 'Ederson',
 'Rodri',
 'Rúben Dias',
 'João Cancelo',
 'İlkay Gündoğan',
 'Gabriel Jesus',
 'Ferrán Torres',
 'Bernardo Silva',
 'Fernandinho',
 'Eric García',
 'Matty Cash',
 'Douglas Luiz',
 'Ezri Konsa',
 'Trézéguet',
 'Ben Chilwell',
 'Thiago Silva',
 'Mateo Kovačić',
 'Jorginho',
 'Emerson Palmieri',
 'Oriol Romeu',
 'Dan Nlundulu',
 'Łukasz Fabiański',
 'Tomáš Souček',
 'Vladimír Coufal',
 'Felipe Anderson',
 'Rui Patrício',
 'Pedro Neto',
 'Daniel Podence',
 'Nélson Semedo',
 'Rúben Neves',
 'João Moutinho',
 'Rayan Aït Nouri',
 'Fábio Silva',
 'Rúben Vinagre',
 'Vitinha',
 'Rodrigo',
 'Pablo Hernández',
 

In [74]:
from fuzzywuzzy import process

In [75]:
# need to create a dictionary for players that don't match
fuzzy_matches = [process.extractOne(x,standard_players)[0] for x in unmatched_players] 
match_list = dict(zip(unmatched_players, fuzzy_matches))
match_list

{'Fabinho': 'Fabio Borini',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Thiago',
 'Allan': 'Adam Lallana',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernardo Espinosa Zúñiga',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bernardo Fernandes da Silva Junior',
 'Fred': 'Fredrik Ulvestad',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Oxlade-Chamberlain',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Ayotomiwa Dele-Bashiru',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de Moraes',
 'Rodri': 'Jay Rodriguez',
 'Rúben D

In [76]:
# some manual corrections
match_list['Fabinho'] = 'Fabio Henrique Tavares'
match_list['Bruno Fernandes'] = 'Bruno Miguel Borges Fernandes'
match_list['Thiago Alcántara'] = 'Thiago Alcántara do Nascimento'
match_list['Fred'] = 'Frederico Rodrigues de Paula Santos'
match_list['Alex Telles'] = 'Alex Nicolao Telles'
match_list['Allan'] = 'Allan Marques Loureiro'
match_list['Bernard'] = 'Bernard Anício Caldeira Duarte'
match_list['Ben Chilwell'] = 'Benjamin Chilwell'
match_list['Jorginho'] = 'Jorge Luiz Frello Filho'
match_list['Dele Alli'] = 'Bamidele Alli'
match_list['Dan Nlundulu'] = "Daniel N'Lundulu"
match_list['Rodri'] = 'Rodrigo Hernandez'
match_list['Rúben Dias'] = 'Rúben Santos Gato Alves Dias'
match_list['João Cancelo'] = 'João Pedro Cavaco Cancelo'
match_list['Gabriel Jesus'] = 'Gabriel Fernando de Jesus'
match_list['Bernardo Silva'] = 'Bernardo Mota Veiga de Carvalho e Silva'
match_list['Fernandinho'] = 'Fernando Luiz Rosa'
match_list['Trézéguet'] = 'Mahmoud Ahmed Ibrahim Hassan'
match_list['Rui Patrício'] = 'Rui Pedro dos Santos Patrício'
match_list['Pedro Neto'] = 'Pedro Lomba Neto'
match_list['Daniel Podence'] = 'Daniel Castelo Podence'
match_list['João Moutinho'] = 'João Filipe Iria Santos Moutinho'
match_list['Rúben Vinagre'] = 'Rúben Gonçalo Silva Nascimento Vinagre'
match_list['Vitinha'] = 'Vitor Ferreira'
match_list['Rodrigo'] = 'Rodrigo Moreno'
match_list['Gabriel Dos Santos'] = 'Gabriel Magalhães'
match_list['Bernardo'] = 'Bernardo Fernandes da Silva Junior'
match_list['Semi Ajayi'] = 'Oluwasemilogo Adesewo Ibidapo Ajayi'
match_list['Angeliño'] = 'Angelo Ogbonna'
match_list['Pedro'] = 'Pedro Rodríguez Ledesma'
match_list['Ricardo Pereira'] = 'Ricardo Domingos Barbosa Pereira'
match_list['Gedson Fernandes'] = 'Gedson Carvalho Fernandes'
match_list['Jonny Castro'] = 'Jonathan Castro Otto'
match_list['Bruno Jordão'] = 'Bruno André Cavaco Jordao'
match_list['Danny Drinkwater'] = 'Daniel Drinkwater'
match_list['Roberto'] = 'Roberto Jimenez Gago'
match_list['Jota'] = 'José Ignacio Peleteiro Romallo'
match_list['João Pedro'] = 'João Pedro Junqueira de Jesus'
match_list['Xande Silva'] = 'Alexandre Nascimento Costa Silva'
match_list['Joselu'] = 'Jose Luis Mato Sanmartín'
match_list['Bruno'] = 'Bruno Saltor Grau'
match_list['Jesé'] = 'Jesé Rodríguez Ruiz'

In [77]:
match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [32]:
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [35]:
season_match_lists = []
seasons = df_enhanced['season'].unique()

for season in seasons:
    
    enhanced_players =  df_enhanced[df_enhanced['season'] == season]['alt_name'].unique().tolist()
    standard_players = df_train[df_train['season'] == season[2:4] + season[7:9]]['player'].unique().tolist()

    unmatched_players = [x for x in enhanced_players if x not in standard_players]
    
    fuzzy_matches = [process.extractOne(x,standard_players)[0] for x in unmatched_players] 
    season_match_lists.append(dict(zip(unmatched_players, fuzzy_matches)))

for season, season_match_list in zip(seasons, season_match_lists):
    print(season)
    print(season_match_list)

2020-2021
{'Ahmed Elmohamady': 'Ahmed El Mohamady', 'Hélder Costa': 'Hélder Wander Sousa de Azevedo e Costa', 'Mohamed Elneny': 'Mohamed Naser El Sayed Elneny', 'Emile Smith-Rowe': 'Emile Smith Rowe', 'Ivan Cavaleiro': 'Ivan Ricardo Neves Abreu Cavaleiro', 'Bobby Reid': 'Bobby Decordova-Reid', 'Ahmed El-Sayed Hegazi': 'Ahmed El-Sayed Hegazy'}
2019-2020
{'Emile Smith-Rowe': 'Emile Smith Rowe', 'Ahmed Elmohamady': 'Ahmed El Mohamady', 'Borja Bastón': 'Borja González Tomás', 'Isaac Success': 'Isaac Success Ajayi'}
2018-2019
{'Léo Bonatini': 'Bonatini Lohner Maia Bonatini', 'Adrien Silva': 'Adrien Sebastian Perruchet Silva', 'Isaac Success': 'Isaac Success Ajayi', 'Josh Sims': 'Joshua Sims', 'José Izquierdo': 'José Diogo Dalot Teixeira', 'Aleksandar Mitrović': 'Aleksandar Mitrovic'}
2017-2018
{'Muhamed Bešić': 'Muhamed Besic', 'Adrien Silva': 'Bernardo Mota Veiga de Carvalho e Silva', 'Aleksandar Mitrović': 'Aleksandar Mitrovic', 'José Izquierdo': 'José Heriberto Izquierdo Mena', 'Josh Sim

In [66]:
# more manual adjustments
season_match_lists[2]['José Izquierdo'] = 'José Heriberto Izquierdo Mena'
season_match_lists[3]['Adrien Silva'] = 'Adrien Sebastian Perruchet Silva'

In [67]:
for season, season_match_list in zip(seasons, season_match_lists):
    print(season)
    print(season_match_list)

2020-2021
{'Ahmed Elmohamady': 'Ahmed El Mohamady', 'Hélder Costa': 'Hélder Wander Sousa de Azevedo e Costa', 'Mohamed Elneny': 'Mohamed Naser El Sayed Elneny', 'Emile Smith-Rowe': 'Emile Smith Rowe', 'Ivan Cavaleiro': 'Ivan Ricardo Neves Abreu Cavaleiro', 'Bobby Reid': 'Bobby Decordova-Reid', 'Ahmed El-Sayed Hegazi': 'Ahmed El-Sayed Hegazy'}
2019-2020
{'Emile Smith-Rowe': 'Emile Smith Rowe', 'Ahmed Elmohamady': 'Ahmed El Mohamady', 'Borja Bastón': 'Borja González Tomás', 'Isaac Success': 'Isaac Success Ajayi'}
2018-2019
{'Léo Bonatini': 'Bonatini Lohner Maia Bonatini', 'Adrien Silva': 'Adrien Sebastian Perruchet Silva', 'Isaac Success': 'Isaac Success Ajayi', 'Josh Sims': 'Joshua Sims', 'José Izquierdo': 'José Heriberto Izquierdo Mena', 'Aleksandar Mitrović': 'Aleksandar Mitrovic'}
2017-2018
{'Muhamed Bešić': 'Muhamed Besic', 'Adrien Silva': 'Adrien Sebastian Perruchet Silva', 'Aleksandar Mitrović': 'Aleksandar Mitrovic', 'José Izquierdo': 'José Heriberto Izquierdo Mena', 'Josh Sims':

In [68]:
for season, season_match_list in zip(seasons, season_match_lists):
    df_enhanced.loc[df_enhanced['season'] == season, 
                    'alt_name'] = df_enhanced[df_enhanced['season'] == season]['alt_name'].replace(season_match_list)

In [78]:
for season in seasons:
    
    enhanced_players =  df_enhanced[df_enhanced['season'] == season]['alt_name'].unique().tolist()
    standard_players = df_train[df_train['season'] == season[2:4] + season[7:9]]['player'].unique().tolist()

    unmatched_players = [x for x in enhanced_players if x not in standard_players]
    print(unmatched_players)

[]
[]
[]
[]


In [79]:
# we'll just need a player name match list for the current season
# so add the 20/21 version to the global version and use this
match_list.update(season_match_lists[0])

In [80]:
match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [81]:
import json

json_file = json.dumps(match_list)
f = open(path/"match_list.json","w")
f.write(json_file)
f.close()

In [82]:
with open(path/'match_list.json') as f:
  test_match_list = json.load(f)

test_match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [46]:
df_train['player'][(df_train['team'] == 'Leicester City')].unique()

array(['Ahmed Musa', 'Andy King', 'Bartosz Kapustka', 'Benjamin Chilwell',
       'Christian Fuchs', 'Daniel Amartey', 'Daniel Drinkwater',
       'Danny Simpson', 'Demarai Gray', 'Gökhan Inler', 'Jamie Vardy',
       'Jeffrey Schlupp', 'Kasper Schmeichel', 'Leonardo Ulloa',
       'Luis Hernández', 'Marcin Wasilewski', 'Marc Albrighton',
       'Nampalys Mendy', 'Ritchie de Laet', 'Riyad Mahrez', 'Robert Huth',
       'Ron-Robert Zieler', 'Shinji Okazaki', 'Wes Morgan',
       'Yohan Benalouane', 'Islam Slimani', 'Ben Hamer', 'Matthew James',
       'Wilfred Ndidi', 'Molla Wagué', 'Elliott Moore', 'Eldin Jakupovic',
       'Harry Maguire', 'Kelechi Iheanacho', 'Matty James',
       'Vicente Iborra', 'Aleksandar Dragovic', 'Hamza Choudhury',
       'George Thomas', 'Adrien Sebastian Perruchet Silva',
       'Fousseni Diabaté', 'Harvey Barnes', 'Sam Hughes', 'Layton Ndukwu',
       'Çaglar Söyüncü', 'Danny Ward', 'Filip Benkovic', 'James Maddison',
       'Jonny Evans', 'Rachid Ghezzal'

In [37]:
df_enhanced[df_enhanced['player'] == 'Alisson']

,season,player,date,squad,comp,shots_total,shots_on_target,touches,pressures,tackles,...,gca,passes_completed,passes,passes_pct,passes_progressive_distance,carries,carry_progressive_distance,dribbles_completed,dribbles,alt_name
86,2020-2021,Alisson,2020-09-12,Liverpool,Premier League,0,0,30,1,0,...,0,19,29,65.5,287,13,20,0,0,Alisson Ramses Becker
87,2020-2021,Alisson,2020-09-20,Liverpool,Premier League,0,0,26,0,0,...,0,24,24,100.0,329,11,12,0,0,Alisson Ramses Becker
88,2020-2021,Alisson,2020-09-28,Liverpool,Premier League,0,0,45,0,0,...,0,35,41,85.4,550,28,31,0,0,Alisson Ramses Becker
89,2020-2021,Alisson,2020-10-24,Liverpool,Premier League,0,0,39,0,0,...,0,25,35,71.4,345,16,31,0,0,Alisson Ramses Becker
90,2020-2021,Alisson,2020-10-31,Liverpool,Premier League,0,0,20,0,0,...,0,16,20,80.0,214,12,43,0,0,Alisson Ramses Becker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15318,2018-2019,Alisson,2019-04-14,Liverpool,Premier League,0,0,35,0,0,...,0,29,33,87.9,468,23,33,0,0,Alisson Ramses Becker
15319,2018-2019,Alisson,2019-04-21,Liverpool,Premier League,0,0,25,0,0,...,0,21,22,95.5,225,12,41,0,0,Alisson Ramses Becker
15320,2018-2019,Alisson,2019-04-26,Liverpool,Premier League,0,0,43,0,0,...,0,39,42,92.9,451,32,33,0,0,Alisson Ramses Becker
15321,2018-2019,Alisson,2019-05-04,Liverpool,Premier League,0,0,31,0,0,...,0,26,29,89.7,497,16,22,0,0,Alisson Ramses Becker


In [83]:
df_enhanced = df_enhanced.drop_duplicates()

In [84]:
df_enhanced.to_csv(path/'enhanced_data.csv')

In [85]:
duplicated = df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]
duplicated

,Unnamed: 0,season,player,date,squad,comp,shots_total,shots_on_target,touches,pressures,...,gca,passes_completed,passes,passes_pct,passes_progressive_distance,carries,carry_progressive_distance,dribbles_completed,dribbles,alt_name


In [86]:
df_train['date'] = df_train['kickoff_time'].str[:10]

In [87]:
df_train[df_train.duplicated(subset=['date','player'])]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date


In [88]:
df_train_further = df_train.merge(df_enhanced.drop(['player', 'season'], axis=1), 
                                  left_on=['date', 'player'], 
                                  right_on=['date', 'alt_name'], how='left')

In [89]:
df_train_further = df_train_further.drop(['alt_name'], axis=1)

In [90]:
assert(len(df_train_further) == len(df_train))

In [91]:
df_train_further.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99072 entries, 0 to 99071
Data columns (total 61 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   player                                      99072 non-null  object 
 1   gw                                          99072 non-null  int64  
 2   position                                    99072 non-null  int64  
 3   minutes                                     99072 non-null  int64  
 4   team                                        99072 non-null  object 
 5   opponent_team                               99072 non-null  object 
 6   relative_market_value_team                  31009 non-null  float64
 7   relative_market_value_opponent_team         30986 non-null  float64
 8   was_home                                    99072 non-null  bool   
 9   total_points                                99072 non-null  int64  
 10  assists   

In [92]:
df_train_further[df_train_further['player'] == 'Mohamed Salah'].iloc[-1]

player                         Mohamed Salah
gw                                        15
position                                   3
minutes                                   90
team                               Liverpool
                                   ...      
passes_progressive_distance              140
carries                                   51
carry_progressive_distance               187
dribbles_completed                         0
dribbles                                   1
Name: 98892, Length: 61, dtype: object

In [93]:
df_train_further.to_csv(path/'train_v7.csv')